# Team 8 : Assignment 4 (OvO Classifier)

## Headers

In [5]:
# all imports here
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import itertools
import argparse
import matplotlib.pyplot as plt
import seaborn as sns

## Importing CSV

In [4]:
# importing dataset from csv file
og_train_data = pd.read_csv("Customer_train.csv")
og_test_data = pd.read_csv("Customer_test.csv")


"""
1838 rows for Training data
790 rows for Testing data
"""

print("Do something here")

Do something here
